### 生成数据集 

In [1]:
import numpy
import tensorflow as tf

In [2]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

In [3]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

2023-02-12 23:35:29.704706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 读取数据集 

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    '''
    Parameters\n
    - is_train : 是否在每个迭代周期内打乱顺序 
    '''
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays) # 对传入的tensor按第一维进行切分
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size) # 划分batch
    return dataset


In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[ 0.5320341 ,  0.02799136],
        [-0.73381555, -0.11618698],
        [ 0.13013203, -0.6122578 ],
        [-0.48618963, -2.3468962 ],
        [ 0.44685557, -0.7927709 ],
        [ 1.5277237 ,  0.25963444],
        [-0.85528094,  0.6727615 ],
        [-1.5420169 , -1.6389347 ],
        [-0.07897481, -1.2461962 ],
        [ 0.5134058 ,  0.13965249]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 5.1575274 ],
        [ 3.121999  ],
        [ 6.554395  ],
        [11.212632  ],
        [ 7.7815857 ],
        [ 6.3626666 ],
        [ 0.21557117],
        [ 6.6804023 ],
        [ 8.272529  ],
        [ 4.779419  ]], dtype=float32)>)

###  定义模型

In [7]:
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1)) # 1是指dense层的输出形状

### 初始化模型参数

In [8]:
initializer = tf.initializers.RandomNormal(stddev=0.01) # 用均值为0，标准差为0.01的随机采样初始化参数
# 在创建层的时候初始化参数
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

### 定义损失函数

In [9]:
loss = tf.keras.losses.MeanSquaredError()

### 定义优化算法

In [10]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

### 训练

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True),y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables)) 
    l = loss(net(X), y)
    print('epoch{},loss:{}'.format(epoch+1, float(l)))


epoch1,loss:0.0002456999500282109
epoch2,loss:8.729896217118949e-05
epoch3,loss:0.00012176130258012563


In [15]:
w = net.get_weights()[0]
b = net.get_weights()[1]
print('w的估计误差为{}'.format(true_w-tf.reshape(w, true_w.shape)))
print('b的估计误差为{}'.format(true_b-b))

w的估计误差为[ 0.00066113 -0.00019908]
b的估计误差为[-0.00031137]
